In [1]:
using LinearAlgebra
using Plots
using LaTeXStrings
using Printf
using Base.Threads
using Optim
using Statistics
using Zygote

# --- 物理パラメータと関数 ---
const KAPPA_MAG = 1.31e-5
const WAVELENGTH_CENTER = 1.031
const TEMP_C = 70.0
const NUM_DOMAINS = 1000

const SELLMEIER_PARAMS = (
    a=[4.5615, 0.08488, 0.1927, 5.5832, 8.3067, 0.021696],
    b=[4.782e-07, 3.0913e-08, 2.7326e-08, 1.4837e-05, 1.3647e-07]
)

function sellmeier_n_eff(wl::Float64, temp::Float64)
    f = (temp - 24.5) * (temp + 24.5 + 2.0 * 273.16)
    lambda_sq = wl^2
    a, b = SELLMEIER_PARAMS.a, SELLMEIER_PARAMS.b
    n_sq = (a[1] + b[1] * f +
            (a[2] + b[2] * f) / (lambda_sq - (a[3] + b[3] * f)^2) +
            (a[4] + b[4] * f) / (lambda_sq - (a[5] + b[5] * f)^2) -
            a[6] * lambda_sq)
    return sqrt(n_sq)
end

function calculate_delta_ks(wl::Float64, temp::Float64)
    wl_fw, wl_shg = wl, wl / 2.0
    n_fw, n_shg = sellmeier_n_eff(wl_fw, temp), sellmeier_n_eff(wl_shg, temp)
    delta_k1 = 2.0 * pi * (n_shg / wl_shg - 2.0 * n_fw / wl_fw)
    wl_thg = (wl_fw * wl_shg) / (wl_fw + wl_shg)
    n_thg = sellmeier_n_eff(wl_thg, temp)
    delta_k2 = 2.0 * pi * (n_thg / wl_thg - n_shg / wl_shg - n_fw / wl_fw)
    return delta_k1, delta_k2
end

const DELTA_K1_GLOBAL, DELTA_K2_GLOBAL = calculate_delta_ks(WAVELENGTH_CENTER, TEMP_C)
const L_GLOBAL = 1im .* [0.0, DELTA_K1_GLOBAL, DELTA_K1_GLOBAL + DELTA_K2_GLOBAL]

A_from_B(B, z, delta_k1, delta_k2) = [B[1], B[2] * exp(-1im * delta_k1 * z), B[3] * exp(-1im * (delta_k1 + delta_k2) * z)]
phi(omega, h) = abs(omega) < 1e-9 ? h + h^2 * omega / 2.0 : (exp(omega * h) - 1.0) / omega

function predictor_ipm1(B_in, h, kappa_val, L)
    B1n, B2n, B3n = B_in
    L1, L2, L3 = L
    exp_Lh = exp.(L .* h)
    omega_a = L2 - 2 * L1
    omega_b = L3 - L2 - L1
    delta_B_NL1 = 1im * kappa_val * exp_Lh[1] * (conj(B1n) * B2n * phi(omega_a, h) + conj(B2n) * B3n * phi(omega_b, h))
    delta_B_NL2 = 1im * kappa_val * exp_Lh[2] * (B1n^2 * phi(-omega_a, h) + 2 * conj(B1n) * B3n * phi(omega_b, h))
    delta_B_NL3 = 1im * 3 * kappa_val * exp_Lh[3] * (B1n * B2n * phi(-omega_b, h))
    return (exp_Lh .* B_in) + [delta_B_NL1, delta_B_NL2, delta_B_NL3]
end


# ★変更点: (N, 2) の superlattice 行列を受け取るように修正
function run_simulation_precomputed(superlattice::Matrix{Float64}, delta_k1::Float64, delta_k2::Float64)
    B = ComplexF64[1.0, 0.0, 0.0]
    L = 1im .* [0.0, delta_k1, delta_k1 + delta_k2]

    num_domains = size(superlattice, 1)
    for i in 1:num_domains
        h = superlattice[i, 1]      # i番目のドメイン幅
        kappa = superlattice[i, 2]  # i番目の非線形係数
        B = predictor_ipm1(B, h, kappa, L)
    end

    # 全長は幅の列の合計
    z_final = sum(superlattice[:, 1])
    A_final = A_from_B(B, z_final, delta_k1, delta_k2)
    return abs2(A_final[2])
end

# ------------------------------------------------------------------
# --- 設計に基づいた構造 (initialize, create_plan は変更なし) ---
# ------------------------------------------------------------------

"""
計算に必要な全ての不変パラメータを格納する構造体
"""
struct SimulationParameters
    wavelengths::Vector{Float64}
    delta_k1s::Vector{Float64}     # 事前計算された delta_k1 の配列
    delta_k2s::Vector{Float64}     # 事前計算された delta_k2 の配列
    temp::Float64
    kappa_mag::Float64
end

"""
1. パラメータ初期化関数 (initialize)
   シミュレーションのパラメータを設定し、delta_kなどを事前計算する。
"""
function initialize()
    wavelength_center = 1.031
    wavelength_span = 0.01
    num_points = 64
    temp = 70.0

    wavelengths = range(wavelength_center - wavelength_span / 2,
        wavelength_center + wavelength_span / 2,
        length=num_points)

    # delta_kを各波長について事前計算
    delta_k1s = zeros(num_points)
    delta_k2s = zeros(num_points)
    for i in 1:num_points
        delta_k1s[i], delta_k2s[i] = calculate_delta_ks(wavelengths[i], temp)
    end

    return SimulationParameters(
        collect(wavelengths),
        delta_k1s,
        delta_k2s,
        temp,
        1.31e-5 # KAPPA_MAG
    )
end

"""
2. 目的関数を生成する高階関数 (create_objective)
   superlattice のみ引数に取る目的関数を返すように修正
"""
function create_objective(params::SimulationParameters)
    function objective_broadband(superlattice::Matrix{Float64})
        num_wavelengths = length(params.wavelengths)
        intensities = zeros(Float64, num_wavelengths)

        Threads.@threads for i in 1:num_wavelengths
            intensities[i] = run_simulation_precomputed(
                superlattice,
                params.delta_k1s[i],
                params.delta_k2s[i]
            )
        end
        return -mean(intensities)
    end
    return objective_broadband
end


function create_plan(objective_func)
    # Zygoteのおかげで、入力がベクトルから行列に変わってもこの関数自体は変更不要
    grad_func! = (g, w) -> (g .= Zygote.gradient(objective_func, w)[1])
    return grad_func!
end


# --- メイン実行部 ---
function main()
    println("Number of threads: ", nthreads())

    # --- 1. 初期化 ---
    params = initialize()
    println("Parameters initialized for $(length(params.wavelengths)) wavelengths.")
    println("-"^50)

    # --- 2. 目的関数と勾配プランの生成 ---
    objective = create_objective(params)
    plan = create_plan(objective)

    # --- 3. 最適化の準備 (superlattice用に変更) ---
    num_domains = 1000
    delta_k1_center, _ = calculate_delta_ks(1.031, params.temp)
    shg_period_center = pi / delta_k1_center

    # 初期 superlattice (N x 2) を作成
    initial_superlattice = zeros(num_domains, 2)
    # 幅(h)の初期値: 理論周期にノイズを加える
    initial_superlattice[:, 1] = fill(shg_period_center, num_domains) .* (1.0 .+ 0.1 .* (rand(num_domains) .- 0.5))
    # 係数(kappa)の初期値: 符号を交互に入れ替える (通常のQPM)
    initial_superlattice[:, 2] = params.kappa_mag .* (-1.0) .^ (1:num_domains)

    # 境界条件 (N x 2) を作成
    lower_bounds = zeros(num_domains, 2)
    upper_bounds = zeros(num_domains, 2)
    # 幅(h)の境界
    lower_bounds[:, 1] .= 0.1
    upper_bounds[:, 1] .= 50.0
    # 係数(kappa)の境界
    lower_bounds[:, 2] .= -params.kappa_mag * 1.1
    upper_bounds[:, 2] .= params.kappa_mag * 1.1


    # --- 4. 最適化の実行 ---
    println("Starting L-BFGS optimization on Superlattice (h, kappa)...")
    optimizer = Fminbox(LBFGS())
    options = Optim.Options(show_trace=true, iterations=100, g_tol=1e-8)

    @time result = optimize(objective, plan, lower_bounds, upper_bounds, initial_superlattice, optimizer, options)

    # --- 5. 結果の表示と可視化 ---
    println("\nOptimization finished.")
    optimized_superlattice = Optim.minimizer(result)

    # 最適化された幅とkappaをそれぞれ取り出す
    optimized_widths = optimized_superlattice[:, 1]
    optimized_kappas = optimized_superlattice[:, 2]

    # 幅の分布をプロット
    p_h = histogram(optimized_widths, bins=50, normalize=:probability,
        label="Optimized Domain Widths",
        xlabel="Domain Width (μm)",
        ylabel="Frequency",
        title="Distribution of Optimized Domain Widths",
        legend=:topright, framestyle=:box)
    vline!(p_h, [shg_period_center], label="Theoretical SHG Period", color=:red, linestyle=:dash, linewidth=2)

    # Kappaの分布をプロット
    p_k = histogram(optimized_kappas, bins=50, normalize=:probability,
        label="Optimized Kappa Values",
        xlabel="Nonlinear Coefficient (kappa)",
        ylabel="Frequency",
        title="Distribution of Optimized Kappa Values",
        legend=:topright, framestyle=:box)
    vline!(p_k, [params.kappa_mag, -params.kappa_mag], label="Initial |Kappa|", color=:red, linestyle=:dash, linewidth=2)

    # 2つのプロットを並べて表示
    display(plot(p_h, p_k, layout=(2, 1)))

end

main()

Number of threads: 64
Parameters initialized for 64 wavelengths.
--------------------------------------------------
Starting L-BFGS optimization on Superlattice (h, kappa)...


BoundsError: BoundsError: attempt to access 0-element Vector{Any} at index []